In [100]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn import set_config
set_config(display="diagram")
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve, roc_auc_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import plot_roc_curve, auc,roc_curve,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [101]:
reviews_df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/car_reviews.csv')

In [102]:
reviews_df = reviews_df.drop(columns=['Unnamed: 0'])

In [103]:
reviews_df

,Car Name,Rating,Reviews
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t..."
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn..."
2,2017 Subaru Impreza 2.0i Sport,5.0,"['November 2, 2022\nBy Sophie from Horse Cave..."
3,2014 Cadillac ATS 2.5L Luxury,4.5,"['May 23, 2023\nBy Helene from Hallandale Beac..."
4,2013 Mercedes-Benz E-Class E 350,4.8,"['March 13, 2021\nBy My personal review from W..."
...,...,...,...
9889,2013 Chevrolet Tahoe LT,1.3,"['May 18, 2021\nBy Mel.R from Boston MA\nOwns ..."
9890,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw..."
9891,2014 Chrysler 200 Limited,4.2,"['May 16, 2023\nBy Coach Joe from Clio MI.\nOw..."
9892,2011 Honda CR-V LX,4.5,"['May 9, 2021\nBy Sheila C from Phoenix AZ\nO..."


In [104]:
car_data_df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/car_data_cleaned.csv')

In [105]:
car_data_df = car_data_df.drop(columns=['Unnamed: 0'])

In [106]:
car_data_df

,Name,Mileage,Dealer Name,Rating,Review Count,Price,Year,Make,Model,Type
0,2015 Ford C-Max Hybrid SEL,"88,380 mi.",Capitol Kia,3.8,(180 reviews),10998.0,2015,Ford C,SEL,Hybrid SEL
1,2016 Kia Sorento LX,"100,837 mi.",Napleton Mazda of Naperville,3.9,(115 reviews),12577.0,2016,Kia Sorento LX,LX,LX
2,2016 Mitsubishi Lancer ES,"133,750 mi.",AVA Auto Sales,0.0,(8 reviews),8495.0,2016,Mitsubishi Lancer ES,ES,ES
3,2011 Ford Escape XLT,"110,862 mi.",94 Nissan Of South Holland,4.7,"(2,682 reviews)",7869.0,2011,Ford Escape XLT,XLT,XLT
4,2017 Dodge Journey SXT,"58,595 mi.",Lithia Chrysler Jeep Dodge Ram of Wasilla,4.6,(543 reviews),14888.0,2017,Dodge Journey SXT,SXT,SXT
...,...,...,...,...,...,...,...,...,...,...
10033,2014 Chevrolet Silverado 1500 LT,"158,598 mi.",City Auto Sales Corp,0.0,(1 review),13995.0,2014,Chevrolet Silverado,LT,1500 LT
10034,2019 Hyundai Sonata SE,"113,581 mi.",Auto Boutique,4.8,"(1,074 reviews)",13500.0,2019,Hyundai Sonata SE,SE,SE
10035,2013 Dodge Journey SXT,"197,496 mi.",Ryan Auto Mall Chrysler Dodge Jeep Ram Of Mont...,0.0,(29 reviews),7425.0,2013,Dodge Journey SXT,SXT,SXT
10036,2007 Lexus ES 350,"85,384 mi.","Mountain Motors, Inc",0.0,(7 reviews),11995.0,2007,Lexus ES,NaN,350


In [107]:
# Making both 'Name' to combine the data
reviews_df.rename(columns={'Car Name': 'Name'}, inplace=True)

In [108]:
# Having the mileage column in float rather than string
car_data_df['Mileage'] = car_data_df['Mileage'].str.replace(' mi.', '').str.replace(',', '')
car_data_df['Mileage'] = pd.to_numeric(car_data_df['Mileage'], errors='coerce')
print(car_data_df['Mileage'])

0         88380
1        100837
2        133750
3        110862
4         58595
          ...  
10033    158598
10034    113581
10035    197496
10036     85384
10037    133800
Name: Mileage, Length: 10038, dtype: int64


In [109]:
# Having Review count to float
car_data_df['Review Count'] = car_data_df['Review Count'].str.replace(r'\D', '', regex=True).astype(float)
print(car_data_df['Review Count'])

0         180.0
1         115.0
2           8.0
3        2682.0
4         543.0
          ...  
10033       1.0
10034    1074.0
10035      29.0
10036       7.0
10037      25.0
Name: Review Count, Length: 10038, dtype: float64


# Merge Dataframes

In [110]:
merged_df = pd.merge(car_data_df, reviews_df_cleaned, on='Name', how='left')

In [111]:
merged_df

,Name,Mileage,Dealer Name,Rating_x,Review Count,Price,Year,Make,Model,Type,Rating_y,Reviews
0,2015 Ford C-Max Hybrid SEL,88380,Capitol Kia,3.8,180.0,10998.0,2015,Ford C,SEL,Hybrid SEL,NaN,NaN
1,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX,4.6,"['July 15, 2023\nBy Mom of three boys from De..."
2,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX,4.6,"['July 15, 2023\nBy Mom of three boys from De..."
3,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX,4.8,"['July 15, 2023\nBy Mom of three boys from De..."
4,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX,5.0,"['July 15, 2023\nBy Mom of three boys from De..."
...,...,...,...,...,...,...,...,...,...,...,...,...
27164,2019 Hyundai Sonata SE,113581,Auto Boutique,4.8,1074.0,13500.0,2019,Hyundai Sonata SE,SE,SE,4.8,"['January 25, 2023\nBy OldEagle from Buffalo N..."
27165,2013 Dodge Journey SXT,197496,Ryan Auto Mall Chrysler Dodge Jeep Ram Of Mont...,0.0,29.0,7425.0,2013,Dodge Journey SXT,SXT,SXT,4.4,"['September 24, 2022\nBy Sabrina Smith from K..."
27166,2013 Dodge Journey SXT,197496,Ryan Auto Mall Chrysler Dodge Jeep Ram Of Mont...,0.0,29.0,7425.0,2013,Dodge Journey SXT,SXT,SXT,4.4,"['September 24, 2022\nBy Sabrina Smith from K..."
27167,2007 Lexus ES 350,85384,"Mountain Motors, Inc",0.0,7.0,11995.0,2007,Lexus ES,NaN,350,NaN,NaN


In [112]:
# Check for duplicate rows in merged_df
duplicate_rows = merged_df[merged_df.duplicated()]

# Print the duplicate rows
print(duplicate_rows)

# Check the number of rows in the merged DataFrame
print("Number of rows in merged_df:", len(merged_df))

# Check the number of rows in the individual DataFrames
print("Number of rows in cars_data_df:", len(car_data_df))
print("Number of rows in reviews_df:", len(reviews_df))

                                    Name  Mileage  \
2                    2016 Kia Sorento LX   100837   
6              2016 Mitsubishi Lancer ES   133750   
7              2016 Mitsubishi Lancer ES   133750   
9                   2011 Ford Escape XLT   110862   
15          2020 Mitsubishi Mirage G4 ES    17531   
...                                  ...      ...   
27145    2016 Jeep Compass High Altitude    88553   
27147           2018 Hyundai Elantra SEL    81189   
27158  2011 Mazda Mazda3 s Grand Touring   121834   
27164             2019 Hyundai Sonata SE   113581   
27166             2013 Dodge Journey SXT   197496   

                                             Dealer Name  Rating_x  \
2                           Napleton Mazda of Naperville       3.9   
6                                         AVA Auto Sales       0.0   
7                                         AVA Auto Sales       0.0   
9                             94 Nissan Of South Holland       4.7   
15           

In [113]:
# Remove duplicate rows from merged_df
merged_df = merged_df.drop_duplicates()

# Check the number of rows in the cleaned merged DataFrame
print("Number of rows in cleaned merged_df:", len(merged_df))

Number of rows in cleaned merged_df: 16852


In [114]:
# Check for duplicate reviews in 'reviews_df'
duplicate_reviews = merged_df.duplicated(subset='Reviews', keep=False)

# Count the number of duplicate reviews
num_duplicate_reviews = len(merged_df[duplicate_reviews])

print("Number of duplicate reviews:", num_duplicate_reviews)

Number of duplicate reviews: 16223


In [115]:
# Drop duplicate reviews based on the 'Reviews' column
reviews_df_cleaned = reviews_df.drop_duplicates(subset='Reviews', keep='first')

# Check the number of rows after dropping duplicates
print("Number of rows in cleaned reviews_df:", len(reviews_df_cleaned))

Number of rows in cleaned reviews_df: 2771


In [116]:
# Check for missing values in each column
missing_values = merged_df.isna().sum()

# Display the count of missing values in each column
print(missing_values)

Name               0
Mileage            0
Dealer Name        0
Rating_x           0
Review Count       0
Price              0
Year               0
Make               0
Model            538
Type             245
Rating_y        3154
Reviews         3154
dtype: int64


In [117]:
# Drop rows with missing 'Model' and 'Type' values
merged_df = merged_df.dropna(subset=['Model', 'Type'])

# Drop 'Rating_y' and 'Reviews' columns
merged_df = merged_df.drop(columns=['Rating_y', 'Reviews'])

# Check the count of missing values again to confirm
missing_values = merged_df.isna().sum()
print(missing_values)

Name            0
Mileage         0
Dealer Name     0
Rating_x        0
Review Count    0
Price           0
Year            0
Make            0
Model           0
Type            0
dtype: int64


In [119]:
merged_df['Model'].fillna('Unknown', inplace=True)
merged_df['Type'].fillna('Unknown', inplace=True)

In [126]:
reviews_df_cleaned

,Name,Rating,Reviews
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t..."
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn..."
2,2017 Subaru Impreza 2.0i Sport,5.0,"['November 2, 2022\nBy Sophie from Horse Cave..."
3,2014 Cadillac ATS 2.5L Luxury,4.5,"['May 23, 2023\nBy Helene from Hallandale Beac..."
4,2013 Mercedes-Benz E-Class E 350,4.8,"['March 13, 2021\nBy My personal review from W..."
...,...,...,...
9859,2013 BMW 535 Gran Turismo i,3.7,"['September 1, 2019\nBy Love BMW from Irvine,..."
9868,2012 BMW 740 i,4.8,"['May 16, 2020\nBy Mike from Los Angeles\nOwns..."
9870,2015 Dodge Charger SE,4.6,"['April 19, 2023\nBy Kathy Lueken from Holden,..."
9874,2011 Lincoln Town Car Signature Limited,4.5,"['June 23, 2023\nBy Eric from Florida\nUsed to..."


# Predicting Rating based on Reviews 


In [129]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X = reviews_df_cleaned['Reviews']
y = reviews_df_cleaned['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Convert text data to TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize the logistic regression model
logreg_model = LogisticRegression()

# Train the model on the TF-IDF vectors
logreg_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = logreg_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.0918918918918919
Classification Report:
              precision    recall  f1-score   support

         1.3       0.00      0.00      0.00         4
         1.5       0.00      0.00      0.00         2
         1.6       0.00      0.00      0.00         1
         1.7       0.00      0.00      0.00         1
         1.8       0.00      0.00      0.00         3
         1.9       0.00      0.00      0.00         6
         2.0       0.00      0.00      0.00         1
         2.1       0.00      0.00      0.00         8
         2.3       0.00      0.00      0.00         2
         2.4       0.00      0.00      0.00         4
         2.5       0.00      0.00      0.00         3
         2.6       0.00      0.00      0.00         2
         2.7       0.00      0.00      0.00         2
         2.8       0.00      0.00      0.00         4
         2.9       0.00      0.00      0.00         3
         3.1       0.00      0.00      0.00         1
         3.2       0.00      

/Users/mahmoud/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [130]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have already loaded the 'reviews_df_cleaned' DataFrame with cleaned data.

# Step 1: Data Balancing (Oversampling)
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(reviews_df_cleaned['Reviews'].values.reshape(-1, 1), reviews_df_cleaned['Rating'].values)

# Convert the resampled data back to a DataFrame
resampled_df = pd.DataFrame({'Reviews': X_resampled.flatten(), 'Rating': y_resampled})

# Step 2: Feature Engineering (TF-IDF Vectorization)
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(resampled_df['Reviews'])

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, resampled_df['Rating'], test_size=0.2, random_state=42)

# Step 4: Model Training (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Model Evaluation
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report_str)


Accuracy: 0.8941717791411042
Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        81
         1.1       1.00      1.00      1.00        57
         1.2       1.00      1.00      1.00        69
         1.3       0.93      1.00      0.97        56
         1.4       1.00      1.00      1.00        64
         1.5       1.00      1.00      1.00        60
         1.6       0.99      1.00      0.99        68
         1.7       1.00      1.00      1.00        63
         1.8       1.00      1.00      1.00        60
         1.9       0.97      1.00      0.98        56
         2.0       1.00      1.00      1.00        60
         2.1       1.00      1.00      1.00        63
         2.2       0.98      1.00      0.99        65
         2.3       1.00      1.00      1.00        65
         2.4       0.88      1.00      0.94        53
         2.5       0.94      1.00      0.97        65
         2.6       1.00      

In [134]:
merged_df

,Name,Mileage,Dealer Name,Rating_x,Review Count,Price,Year,Make,Model,Type
0,2015 Ford C-Max Hybrid SEL,88380,Capitol Kia,3.8,180.0,10998.0,2015,Ford C,SEL,Hybrid SEL
1,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX
3,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX
4,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,115.0,12577.0,2016,Kia Sorento LX,LX,LX
5,2016 Mitsubishi Lancer ES,133750,AVA Auto Sales,0.0,8.0,8495.0,2016,Mitsubishi Lancer ES,ES,ES
...,...,...,...,...,...,...,...,...,...,...
27161,2019 Hyundai Sonata SE,113581,Auto Boutique,4.8,1074.0,13500.0,2019,Hyundai Sonata SE,SE,SE
27162,2019 Hyundai Sonata SE,113581,Auto Boutique,4.8,1074.0,13500.0,2019,Hyundai Sonata SE,SE,SE
27163,2019 Hyundai Sonata SE,113581,Auto Boutique,4.8,1074.0,13500.0,2019,Hyundai Sonata SE,SE,SE
27165,2013 Dodge Journey SXT,197496,Ryan Auto Mall Chrysler Dodge Jeep Ram Of Mont...,0.0,29.0,7425.0,2013,Dodge Journey SXT,SXT,SXT


# NLP

In [66]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_norm = X.apply(self.process_doc)
        return X_norm

    def process_doc(self, doc):
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:
                return None
            
        # remove stop words and punctuations, then lower case
        doc_norm = [tok.lower() for tok in word_tokenize(doc) if tok.isalpha() and tok.lower() not in self.stop_words]
        
        #  POS detection on the result will be important in telling Wordnet's lemmatizer how to lemmatize
        
        # creates list of tuples with tokens and POS tags in wordnet format
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm)))
        doc_norm = [self.wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]
        return " ".join(doc_norm)


# Predict Price with reviews

In [164]:
# Convert the 'Rating' column in reviews_df_cleaned to float
reviews_df_cleaned['Rating'] = reviews_df_cleaned['Rating'].astype(float)

# Step 1: Merge car_data_df with reviews_df_cleaned based on Name and Rating columns
combined_df = pd.merge(car_data_df, reviews_df_cleaned[['Name', 'Rating', 'Reviews']], on=['Name', 'Rating'])


<ipython-input-164-d7b238720d0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_cleaned['Rating'] = reviews_df_cleaned['Rating'].astype(float)


In [165]:
import re
import string

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

combined_df['Processed_Reviews'] = combined_df['Reviews'].apply(preprocess_text)

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(combined_df['Processed_Reviews'])

In [167]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, combined_df['Price'], test_size=0.2, random_state=42)


In [168]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Build the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 10060491.479230678
R-squared: -0.03190421553979861


In [173]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Merge car_data_df with reviews_df_cleaned based on Name and Rating columns
combined_df = pd.merge(car_data_df, reviews_df_cleaned[['Name', 'Rating', 'Reviews']], on=['Name', 'Rating'])

# Step 2: One-hot encode the 'Make', 'Model', and 'Type' columns
combined_df = pd.get_dummies(combined_df, columns=['Make', 'Model', 'Type'], drop_first=True)

# Step 3: Text preprocessing of the 'Reviews' column (if required)
# Assuming you have already defined the 'preprocess_text' function as before
combined_df['Processed_Reviews'] = combined_df['Reviews'].apply(preprocess_text)

# Step 4: TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = vectorizer.fit_transform(combined_df['Processed_Reviews'])

# Step 5: Drop irrelevant columns and combine numerical features with TF-IDF vectors
X = combined_df.drop(['Name', 'Mileage', 'Dealer Name', 'Reviews', 'Processed_Reviews', 'Price'], axis=1)
X_combined = pd.concat([X.reset_index(drop=True), pd.DataFrame(X_tfidf.toarray())], axis=1)

# Step 6: Split the data into training and testing sets
y = combined_df['Price']
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 7: Model Training (Random Forest Regressor)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 8: Model Evaluation
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


Root Mean Squared Error: 2796.5133334852267
R-squared: 0.19785297385408418


In [174]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Step 1: Feature Engineering
combined_df['Car_Age'] = 2023 - combined_df['Year']

# Step 2: Drop irrelevant columns and combine numerical features with TF-IDF vectors
X = combined_df.drop(['Name', 'Mileage', 'Dealer Name', 'Reviews', 'Processed_Reviews', 'Price', 'Year'], axis=1)
X_combined = pd.concat([X.reset_index(drop=True), pd.DataFrame(X_tfidf.toarray())], axis=1)

# Step 3: Split the data into training and testing sets
y = combined_df['Price']
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 4: Model Training (Gradient Boosting Regressor)
gb_model = GradientBoostingRegressor(random_state=42)

# Step 5: Hyperparameter Tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(gb_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Step 6: Model Evaluation
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


Root Mean Squared Error: 2820.6232391365006
R-squared: 0.18396206431196838


In [184]:
combined_df

,Name,Mileage,Dealer Name,Rating,Review Count,Price,Year,Make,Model,Type,Reviews,Age,Car_Age
0,2014 Nissan Altima 2.5 S,107615.0,Mike Rezi Nissan Atlanta,4.9,2341.0,9000.0,2014,Nissan Altima,S,2.5 S,"['November 16, 2022\nBy Andy from Albany, NY\n...",9.0,9
1,2014 Nissan Altima 2.5 S,107615.0,Mike Rezi Nissan Atlanta,4.9,2341.0,9000.0,2014,Nissan Altima,S,2.5 S,"['November 16, 2022\nBy Andy from Albany, NY\n...",9.0,9
2,2014 Nissan Altima 2.5 S,72881.0,AP's Auto Sales,4.9,99.0,10777.0,2014,Nissan Altima,S,2.5 S,"['November 16, 2022\nBy Andy from Albany, NY\n...",9.0,9
3,2006 Toyota Tacoma Access Cab,155548.0,Coggin Honda of Fort Pierce,3.3,661.0,9999.0,2006,Toyota Tacoma Access Cab,Cab,Access Cab,"['March 11, 2023\nBy Juan from La puente ca\nO...",17.0,17
4,2003 Mercedes-Benz C-Class 4dr Sdn 2.6L,206080.0,Fitzgerald Chrysler Jeep Dodge RAM Lexington Park,4.4,457.0,5500.0,2003,Mercedes,6L,4dr Sdn 2.6L,"['July 16, 2023\nBy 300D Man from Hanover Pa\n...",20.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2015 Lincoln MKZ Hybrid Base,145111.0,Beemer Haus,4.7,134.0,8998.0,2015,Lincoln MKZ Hybrid Base,Base,Hybrid Base,"['November 28, 2020\nBy TCB from Mineral Point...",8.0,8
606,2010 Chrysler 300 300S V6,185000.0,Legend Auto Sales Inc.,4.5,18.0,6995.0,2010,Chrysler,V6,300S V6,"['March 18, 2022\nBy Sean from Green Bay, Wisc...",13.0,13
607,2016 Honda CR-Z LX,72789.0,Honda City Chicago,4.7,782.0,14995.0,2016,Honda CR,LX,LX,"['June 13, 2023\nBy Greenbear from Alaska\nOwn...",7.0,7
608,1990 Chevrolet Corvette Base,111110.0,Skyway Classics,4.2,24.0,8997.0,1990,Chevrolet Corvette Base,Base,Base,"['April 3, 2023\nBy Alanna Nugent from Harpe...",33.0,33


In [187]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Step 1: Feature Engineering
combined_df['Car_Age'] = 2023 - combined_df['Year']

# Step 2: Drop irrelevant columns and separate numerical and categorical features
X = combined_df.drop(['Name', 'Dealer Name', 'Reviews', 'Year'], axis=1)
numerical_features = ['Mileage', 'Rating', 'Review Count', 'Age', 'Car_Age']
categorical_features = ['Make', 'Model', 'Type']

# Step 3: One-hot encode the categorical features and combine with numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

X_combined = preprocessor.fit_transform(X)

# Step 4: Split the data into training and testing sets
y = combined_df['Price']
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 5: Model Training (Gradient Boosting Regressor)
gb_model = GradientBoostingRegressor(random_state=42)

# Step 6: Hyperparameter Tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(gb_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Step 7: Model Evaluation
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


Root Mean Squared Error: 2464.2700960144043
R-squared: 0.3771308764453909


In [192]:
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Feature Engineering
combined_df['Car_Age'] = 2023 - combined_df['Year']

# Step 2: Separate numerical, categorical, and text features
numerical_features = ['Rating', 'Review Count', 'Age', 'Car_Age']
categorical_features = ['Make', 'Model', 'Type']
text_feature = 'Reviews'

X_numerical = combined_df[numerical_features]
X_categorical = combined_df[categorical_features]
X_text = combined_df[text_feature]
y = combined_df['Price']

# Step 3: Split the data into training and testing sets
X_train_numerical, X_test_numerical, X_train_categorical, X_test_categorical, X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text, y, test_size=0.2, random_state=42)

# Step 4: Create preprocessing pipelines for numerical and categorical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english'))
])

preprocessor_numerical = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features)
])

preprocessor_categorical = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_features)
])

preprocessor_text = ColumnTransformer([
    ('text', text_pipeline, text_feature)
])

# Step 5: Model Training (XGBoost)
xgb_model = xgb.XGBRegressor(random_state=42)

# Step 6: Hyperparameter Tuning using RandomizedSearchCV (same as before)

# Step 7: Create the final pipeline by combining all preprocessors and the regressor
final_pipeline = Pipeline([
    ('preprocessor_numerical', preprocessor_numerical),
    ('preprocessor_categorical', preprocessor_categorical),
    ('preprocessor_text', preprocessor_text),
    ('regressor', xgb_model)
])

# Step 8: Fit the final pipeline and evaluate the model
final_pipeline.fit(X_train_numerical, y_train)

y_pred = final_pipeline.predict(X_test_numerical)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


ValueError: Specifying the columns using strings is only supported for pandas DataFrames

# Spliting the Data and Using the NLP Class to process the Data

# Vectorization and Supervised Model